In [1]:
import os
import os.path as osp
import cv2
import shutil
import random
from tqdm.notebook import tqdm
from augmentations import augmentation_choices, transform_image

In [ ]:
# In this cell, define a function to save the NEW split files
# Save a split file for EACH augmentation
# and one for the whole augmented dataset

In [6]:
def make_subdir(subdir):
    if not osp.isdir(subdir):
        try:
            os.mkdir(subdir)
        except FileNotFoundError:
            print("Make sure the parent folder exists")

In [16]:
split_file = 'kitti_eigen_test.txt'
input_prefix_path = "E:\Thesis_Datasets\\kitti_full\\"
input_depth_dir = 'val'
valid_image_count = 0
img_infos = []
output_prefix_path = "E:\Thesis_Datasets\\Augmented_Images\\"
output_image_folder = "Data\\"
output_depth_folder = "Depth\\"

with open(split_file) as f:
    for line in tqdm(f, desc="Augmenting input files...", total=697):
        img_name = line.strip().split(" ")[0]
        img_name = osp.join(input_prefix_path, img_name)
        depth_file = line.strip().split(" ")[1]
        depth_map = osp.join(input_prefix_path, input_depth_dir, depth_file)

        if osp.isfile(img_name) and osp.isfile(depth_map):
            valid_image_count += 1

            # Ensure the folder exists
            make_subdir(osp.join(output_prefix_path, output_depth_folder))

            #   Copy depth map
            shutil.copy(depth_map,
                        osp.join(output_prefix_path, output_depth_folder, depth_map.split('/')[-1]))

            #   For each augmentation type:
            for aug in augmentation_choices:
                make_subdir(osp.join(output_prefix_path, output_image_folder, aug))

                # if aug = affine, depth map needs to be augmented with the same scale, rotation and shear
                if aug == 'affine':
                    make_subdir(osp.join(output_prefix_path, output_depth_folder, aug))

                    scale=(1.1, 1.1)
                    rotation=random.uniform(-0.05, 0.05)
                    shear=random.uniform(-0.1, 0.1)

                    img = transform_image(aug, cv2.imread(img_name), scale, rotation, shear)
                    augmented_depth = transform_image(aug, cv2.imread(depth_map), scale, rotation, shear)

                    # Save transformed depth map in Depth/affine/xxxxxxx_affine.png
                    cv2.imwrite(osp.join(output_prefix_path, 
                                     output_depth_folder, 
                                     aug, 
                                     img_name[img_name.rfind('/')+1:-4] + "_affine.png"),
                                augmented_depth)
                    # TODO: insert [rgb_file, depth_file, augmentation_type] into LIST
                    
                else:
                    img = transform_image(aug, cv2.imread(img_name))
                    # insert [rgb_file, depth_file, augmentation_type] into LIST
                    
                # Save file with new name in /<augmentation_type>/ folder
                cv2.imwrite(osp.join(output_prefix_path, 
                                     output_image_folder, 
                                     aug, 
                                     img_name.split('/')[-1]),
                            img)

print(f"Total: {valid_image_count} input files")


Augmenting input files...:   0%|          | 0/697 [00:00<?, ?it/s]

Total: 161 input files


In [19]:
# valid_image_count is 161. Need to figure out why.
valid_image_count

161